In [1]:
# ==============================================
# 🔹 Step 1 — Install dependencies
# ==============================================
!pip install transformers datasets evaluate accelerate -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [2]:
# ==============================================
# 🔹 Step 2 — Import libraries
# ==============================================
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
import torch


In [21]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/drive/MyDrive/spoc-train.tsv",sep='\t')

# Drop empty rows
df = df.dropna(subset=['text', 'code'])

# Convert to string
df['text'] = df['text'].astype(str)
df['code'] = df['code'].astype(str)

# Combine into GPT-2 friendly format
df['combined'] = df.apply(
    lambda x: f"<BOS> Pseudocode: {x['text']} <SEP> Code: {x['code']} <EOS>", axis=1
)

# 2️⃣ Save as .csv (for structured dataset usage)
df[['text', 'code']].rename(columns={'text': 'input_text', 'code': 'target_text'}).to_csv(
    "/content/drive/MyDrive/spoc_processed.csv", index=False
)


In [25]:
# ==============================================
# 🔹 Step 3 — Load preprocessed data
# ==============================================
from google.colab import drive
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/spoc_processed.csv"
df = pd.read_csv(file_path)

print("✅ Loaded data:", len(df), "pairs")
print(df.head(2))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Loaded data: 216225 pairs
                                        input_text  \
0                in the function gcd(a,b=integers)   
1  if b=1 return a, else call function gcd(b, a%b)   

                      target_text  
0         int gcd(int a, int b) {  
1  return !b ? a : gcd(b, a % b);  


In [26]:
# ==============================================
# 🔹 Step 4 — Load GPT-2 tokenizer
# ==============================================
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # important for GPT-2


In [33]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'bos_token': '<BOS>', 'eos_token': '<EOS>', 'sep_token': '<SEP>'})

# 1️⃣ Load your preprocessed text–code file again
df = pd.read_csv("/content/drive/MyDrive/spoc_processed.csv", sep=',', engine='python')

# 2️⃣ Keep only 'text' (pseudocode) and 'code'
df = df[['input_text', 'target_text']].dropna()


# 4️⃣ Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# 5️⃣ Split into training and test data
dataset = dataset.train_test_split(test_size=0.1, seed=42)

# 6️⃣ Define tokenize function
def tokenize_function(example):
    inputs = tokenizer(
        example["input_text"],
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    labels = tokenizer(
        example["target_text"],
        truncation=True,
        padding="max_length",
        max_length=256,
    )["input_ids"]
    inputs["labels"] = labels
    return inputs

# 7️⃣ Map over both splits
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=list(df.columns)
)

print("✅ Tokenization successful!")
print(tokenized_datasets)


Map:   0%|          | 0/194602 [00:00<?, ? examples/s]

Map:   0%|          | 0/21623 [00:00<?, ? examples/s]

✅ Tokenization successful!
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 194602
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21623
    })
})


In [34]:
# ==============================================
# 🔹 Step 6 — Load GPT-2 model
# ==============================================
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


Embedding(50261, 768)

In [35]:

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/gpt2_spoc_finetuned",  # where to save model
    eval_strategy="epoch",  # correct key (not eval_strategy)
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False,
    logging_dir="./logs",

    # 🔹 Disable wandb logging completely
    report_to="none",        # stop huggingface from using wandb
    logging_strategy="epoch" # still log locally each epoch
)


In [38]:
# Use only first 5000 samples for testing fine-tuning
small_train = tokenized_datasets["train"].select(range(2000))
small_test = tokenized_datasets["test"].select(range(500))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_test,
    tokenizer=tokenizer,
)
trainer.train()


/tmp/ipython-input-258188350.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


TrainOutput(global_step=3000, training_loss=0.0993523432413737, metrics={'train_runtime': 673.4702, 'train_samples_per_second': 8.909, 'train_steps_per_second': 4.455, 'total_flos': 783876096000000.0, 'train_loss': 0.0993523432413737, 'epoch': 3.0})

In [39]:
# ==============================================
# 🔹 Step 9 — Save final fine-tuned model
# ==============================================
model.save_pretrained("/content/drive/MyDrive/gpt2_spoc_final")
tokenizer.save_pretrained("/content/drive/MyDrive/gpt2_spoc_final")

print("✅ Model saved successfully!")


✅ Model saved successfully!


In [40]:
from transformers import pipeline

generator = pipeline("text-generation", model="/content/drive/MyDrive/gpt2_spoc_final", tokenizer=tokenizer)

pseudo = """declare integer variable n and print all even numbers from 1 to n"""
result = generator(f"<BOS> Pseudocode: {pseudo} <SEP> Code:", max_length=150, num_return_sequences=1)

print("🧩 Generated Code:\n")
print(result[0]['generated_text'])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🧩 Generated Code:

<BOS> Pseudocode: declare integer variable n and print all even numbers from 1 to n <SEP> Code:erverServer {er } } } {er {er CLIENT } {er {er {er}, } } } } } } } } } } } } } } } } } } } } } } } } } } } { } { } { } { } { } { } { } { } { { } { } { } { } { } { } { } { } { { } { } { } { } { } { { } { } { } { { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } { } {


In [43]:
# ==============================================
# 📦 COMPLETE WORKING VERSION - NO ERRORS
# ==============================================

# Step 1: Install Dependencies
!pip install transformers datasets evaluate accelerate -q

# Step 2: Import Libraries
import pandas as pd
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
import torch
from google.colab import drive

# Step 3: Mount Drive and Load Data
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/spoc-train.tsv", sep='\t')
df = df.dropna(subset=['text', 'code'])
df['text'] = df['text'].astype(str).str.strip()
df['code'] = df['code'].astype(str).str.strip()
# ✅ Reduce dataset for faster training
df = df.sample(n=5000, random_state=42).reset_index(drop=True)

print(f"✅ Loaded {len(df)} examples")

# Step 4: Format Data for Causal LM
df['prompt'] = df.apply(
    lambda x: f"Pseudocode: {x['text']}\nCode: {x['code']}",
    axis=1
)

print("\n✅ Sample formatted prompt:")
print(df['prompt'].iloc[0][:200])

# Step 5: Initialize Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Step 6: Split Data
train_size = int(0.95 * len(df))
train_df = df[:train_size].reset_index(drop=True)
test_df = df[train_size:].reset_index(drop=True)

print(f"\nTrain size: {len(train_df)}")
print(f"Test size: {len(test_df)}")

# Step 7: Tokenization with FIXED padding
def tokenize_and_prepare(examples):
    """Tokenize text and create labels"""
    # Tokenize with consistent padding
    tokenized = tokenizer(
        examples['prompt'],
        truncation=True,
        max_length=256,  # Reasonable length for code
        padding='max_length',
        return_tensors=None  # Keep as lists
    )

    # Create labels (copy of input_ids for causal LM)
    # Replace pad tokens with -100 so they're ignored in loss
    labels = []
    for input_ids in tokenized['input_ids']:
        label_ids = [
            token_id if token_id != tokenizer.pad_token_id else -100
            for token_id in input_ids
        ]
        labels.append(label_ids)

    tokenized['labels'] = labels
    return tokenized

# Step 8: Create and Tokenize Datasets
print("\n🔄 Tokenizing data...")

train_dataset = Dataset.from_pandas(train_df[['prompt']])
test_dataset = Dataset.from_pandas(test_df[['prompt']])

tokenized_train = train_dataset.map(
    tokenize_and_prepare,
    batched=True,
    remove_columns=['prompt'],
    desc="Tokenizing train"
)

tokenized_test = test_dataset.map(
    tokenize_and_prepare,
    batched=True,
    remove_columns=['prompt'],
    desc="Tokenizing test"
)

print("✅ Tokenization complete!")

# Verify shapes
print(f"Input IDs shape: {len(tokenized_train[0]['input_ids'])}")
print(f"Labels shape: {len(tokenized_train[0]['labels'])}")
print(f"Attention mask shape: {len(tokenized_train[0]['attention_mask'])}")

# Step 9: Load Model
print("\n🤖 Loading model...")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

# Step 10: Training Arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/gpt2_pseudocode_v3",
    eval_strategy="steps",
    eval_steps=1000,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=2000,
    logging_steps=200,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=True,
    warmup_steps=500,
    gradient_accumulation_steps=2,
)

# Step 11: Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
)

# Step 12: Train
print("\n🚀 Starting training...")
print("="*60)
trainer.train()

# Step 13: Save Model
final_model_path = "/content/drive/MyDrive/gpt2_pseudocode_final_v3"
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"\n✅ Model saved to {final_model_path}")

# Step 14: Test Generation Function
def generate_code(pseudocode, max_new_tokens=150):
    """Generate code from pseudocode"""
    prompt = f"Pseudocode: {pseudocode}\nCode:"
    inputs = tokenizer(prompt, return_tensors="pt")

    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        model.cuda()

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the code part
    if "Code:" in generated_text:
        code = generated_text.split("Code:", 1)[1].strip()
        return code
    return generated_text

# Step 15: Test with Examples
print("\n" + "="*60)
print("🧪 TESTING CODE GENERATION")
print("="*60)

test_cases = [
    "declare integer variable n and print all even numbers from 1 to n",
    "function to calculate factorial of a number",
    "create a list of squares of numbers from 1 to 10",
    "read a string and count vowels",
]

for i, pseudo in enumerate(test_cases, 1):
    print(f"\n{i}. Pseudocode:")
    print(f"   {pseudo}")
    print("\n   Generated Code:")
    try:
        code = generate_code(pseudo)
        print(f"   {code}")
    except Exception as e:
        print(f"   Error: {e}")
    print("-" * 60)

print("\n✅ All done!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Loaded 5000 examples

✅ Sample formatted prompt:
Pseudocode: else
Code: } else {

Train size: 4750
Test size: 250

🔄 Tokenizing data...


Tokenizing train:   0%|          | 0/4750 [00:00<?, ? examples/s]

Tokenizing test:   0%|          | 0/250 [00:00<?, ? examples/s]

✅ Tokenization complete!
Input IDs shape: 256
Labels shape: 256
Attention mask shape: 256

🤖 Loading model...


/tmp/ipython-input-2114238919.py:132: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.



🚀 Starting training...



✅ Model saved to /content/drive/MyDrive/gpt2_pseudocode_final_v3

🧪 TESTING CODE GENERATION

1. Pseudocode:
   declare integer variable n and print all even numbers from 1 to n

   Generated Code:
   int N, P = {1, maxN; }; cout << "n" >> p.size() % 2f6++; } else if ((mn == 0) cin >> m && !l || (!maxN)) return a[i][j]'; }); break ;;;} void loop while (*mx != y)(int k *p) { long double i - x + ans6432_t(); do the following as values of function strlen(), returning bool : ((long long)strLen());), read vnd); continue; for (_ai | t|){ std::cin in _a ++ jowles((cnt8*k)] > min()) s += b.' '\0' /
------------------------------------------------------------

2. Pseudocode:
   function to calculate factorial of a number

   Generated Code:
   int f = (a * b) + 1; -1; } else { cout << "fact=((b.size() / 2)) % n"; return ans >> endl; })(); --end-of--line}::cin,c++){ cin >> truth(int s) ? '(' : ss && (').char()); while ((s == true || d != 0)); do the following(); break;; }; ++ctime()) {}'; get("F

In [45]:
test_cases = [
    "take 10 numbers input and find prime numbers",

]

for i, pseudo in enumerate(test_cases, 1):
    print(f"\n{i}. Pseudocode:")
    print(f"   {pseudo}")
    print("\n   Generated Code:")
    try:
        code = generate_code(pseudo)
        print(f"   {code}")
    except Exception as e:
        print(f"   Error: {e}")
    print("-" * 60)

print("\n✅ All done!")


1. Pseudocode:
   take 10 numbers input and find prime numbers

   Generated Code:
   for (i = 0; i < n; i++) { cin >> 9999[n]; } else if ((9999[1] - 1) % 2 == '.'); return 1000000007 * 100099998 + 785234715975e5, true ; break} cout << "10" << endl;; // prints 5 newline || "\|O.\N"; while (!cin & j != "-2") pgcd(j, "'", ++dans)) s0<= 3 / dz++; loop ten..>(); lwcntime is equal to tpow((siz()-t)(a)), mx <= ss ? r : arr()) && !r
------------------------------------------------------------

✅ All done!


In [46]:
# Step 16: Evaluate Model (BLEU)
from evaluate import load
bleu = load("bleu")

def evaluate_bleu(dataset, num_samples=50):
    """Evaluate BLEU on a subset of data"""
    references, predictions = [], []

    for i in range(min(num_samples, len(dataset))):
        pseudo = df['text'].iloc[i]
        true_code = df['code'].iloc[i]
        pred_code = generate_code(pseudo)
        references.append([true_code])
        predictions.append(pred_code)

    results = bleu.compute(predictions=predictions, references=references)
    print(f"\n📊 BLEU Score: {results['bleu']:.4f}")

evaluate_bleu(test_df)



📊 BLEU Score: 0.0177


In [50]:
# ============================================================
# 🚀 STREAMLIT APP FOR GPT-2 CODE GENERATION WITH NGROK
# ============================================================
from google.colab import drive
drive.mount('/content/drive')
# Step 1: Install required packages
!pip install streamlit pyngrok transformers torch -q

# Step 2: Create Streamlit app file
app_code = '''import streamlit as st
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import os

# Page config
st.set_page_config(
    page_title="Pseudocode to Code Generator",
    page_icon="🤖",
    layout="wide"
)

# Custom CSS
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
    }
    .subtitle {
        font-size: 1.2rem;
        color: #666;
        text-align: center;
        margin-bottom: 3rem;
    }
    .code-box {
        background-color: #f0f2f6;
        padding: 1.5rem;
        border-radius: 0.5rem;
        border-left: 4px solid #1f77b4;
        font-family: 'Courier New', monospace;
    }
</style>
""", unsafe_allow_html=True)

# Cache model loading
@st.cache_resource
def load_model():
    """Load the fine-tuned GPT-2 model"""
    model_path = "/content/drive/MyDrive/gpt2_pseudocode_final_v3"

    with st.spinner("🔄 Loading model... This may take a moment..."):
        tokenizer = GPT2Tokenizer.from_pretrained(model_path)
        model = GPT2LMHeadModel.from_pretrained(model_path)

        if torch.cuda.is_available():
            model = model.cuda()

    return tokenizer, model

# Code generation function
def generate_code(pseudocode, tokenizer, model, max_tokens=150, temperature=0.8, top_p=0.95):
    """Generate code from pseudocode"""
    prompt = f"Pseudocode: {pseudocode}\\nCode:"
    inputs = tokenizer(prompt, return_tensors="pt")

    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract code part
    if "Code:" in generated_text:
        code = generated_text.split("Code:", 1)[1].strip()
        return code
    return generated_text

# Main app
def main():
    # Header
    st.markdown('<h1 class="main-header">🤖 Pseudocode to Code Generator</h1>', unsafe_allow_html=True)
    st.markdown('<p class="subtitle">Powered by Fine-tuned GPT-2 on SPOC Dataset</p>', unsafe_allow_html=True)

    # Load model
    try:
        tokenizer, model = load_model()
        st.success("✅ Model loaded successfully!")
    except Exception as e:
        st.error(f"❌ Error loading model: {e}")
        st.info("💡 Make sure the model path is correct: `/content/drive/MyDrive/gpt2_pseudocode_final_v3`")
        return

    # Sidebar for settings
    with st.sidebar:
        st.header("⚙️ Generation Settings")

        max_tokens = st.slider(
            "Max Tokens",
            min_value=50,
            max_value=300,
            value=150,
            step=10,
            help="Maximum number of tokens to generate"
        )

        temperature = st.slider(
            "Temperature",
            min_value=0.1,
            max_value=2.0,
            value=0.8,
            step=0.1,
            help="Higher = more creative, Lower = more focused"
        )

        top_p = st.slider(
            "Top P (Nucleus Sampling)",
            min_value=0.1,
            max_value=1.0,
            value=0.95,
            step=0.05,
            help="Probability threshold for nucleus sampling"
        )

        st.markdown("---")
        st.markdown("### 📊 Model Info")
        st.info("""
        - **Base Model**: GPT-2
        - **Training Data**: SPOC Dataset
        - **Task**: Pseudocode → Code
        """)

    # Main content area
    col1, col2 = st.columns(2)

    with col1:
        st.header("📝 Input Pseudocode")

        # Example buttons
        st.markdown("**Quick Examples:**")
        example_col1, example_col2 = st.columns(2)

        with example_col1:
            if st.button("🔢 Even Numbers"):
                st.session_state.pseudocode = "declare integer variable n and print all even numbers from 1 to n"
            if st.button("🔄 Factorial"):
                st.session_state.pseudocode = "function to calculate factorial of a number"

        with example_col2:
            if st.button("📊 Squares List"):
                st.session_state.pseudocode = "create a list of squares of numbers from 1 to 10"
            if st.button("🔤 Count Vowels"):
                st.session_state.pseudocode = "read a string and count vowels"

        # Text input
        pseudocode = st.text_area(
            "Enter your pseudocode:",
            value=st.session_state.get('pseudocode', ''),
            height=200,
            placeholder="Example: declare integer variable n and print all even numbers from 1 to n"
        )

        # Generate button
        generate_btn = st.button("🚀 Generate Code", type="primary", use_container_width=True)

    with col2:
        st.header("💻 Generated Code")

        if generate_btn:
            if not pseudocode.strip():
                st.warning("⚠️ Please enter some pseudocode first!")
            else:
                with st.spinner("🔄 Generating code..."):
                    try:
                        generated_code = generate_code(
                            pseudocode,
                            tokenizer,
                            model,
                            max_tokens=max_tokens,
                            temperature=temperature,
                            top_p=top_p
                        )

                        # Display generated code
                        st.markdown('<div class="code-box">', unsafe_allow_html=True)
                        st.code(generated_code, language="cpp")
                        st.markdown('</div>', unsafe_allow_html=True)

                        # Copy button
                        st.download_button(
                            label="📋 Download Code",
                            data=generated_code,
                            file_name="generated_code.cpp",
                            mime="text/plain"
                        )

                    except Exception as e:
                        st.error(f"❌ Error generating code: {e}")
        else:
            st.info("👈 Enter pseudocode and click 'Generate Code' to see results")

    # Footer
    st.markdown("---")
    st.markdown("""
    <div style="text-align: center; color: #666;">
        <p>Built with ❤️ using Streamlit | Fine-tuned GPT-2 Model</p>
    </div>
    """, unsafe_allow_html=True)

if __name__ == "__main__":
    main()
'''

# Write the app to file
with open('app.py', 'w') as f:
    f.write(app_code)

print("✅ app.py created successfully!")

# Step 3: Setup ngrok authentication
from pyngrok import ngrok, conf
import getpass

print("\n" + "="*60)
print("🔐 NGROK AUTHENTICATION SETUP")
print("="*60)
print("\n1. Go to: https://dashboard.ngrok.com/get-started/your-authtoken")
print("2. Copy your authtoken")
print("3. Paste it below\n")

ngrok_token = getpass.getpass("Enter your ngrok authtoken: ")
ngrok.set_auth_token(ngrok_token)

# Step 4: Start Streamlit with ngrok
print("\n" + "="*60)
print("🚀 STARTING STREAMLIT APP")
print("="*60)

# Kill any existing Streamlit processes
!pkill -9 streamlit

# Start streamlit in background
import subprocess
import time

# Start Streamlit
streamlit_process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for Streamlit to start
time.sleep(5)

# Start ngrok tunnel
public_url = ngrok.connect(8501)

print("\n✅ Streamlit app is running!")
print(f"🌐 Public URL: {public_url}")
print(f"📱 Share this URL with anyone to access your app!")
print("\n⚠️ Note: Keep this cell running. Stopping it will close the app.")
print("="*60)

# Keep the cell running
try:
    streamlit_process.wait()
except KeyboardInterrupt:
    print("\n\n🛑 Shutting down...")
    streamlit_process.terminate()
    ngrok.disconnect(public_url)
    print("✅ App stopped successfully!")

✅ app.py created successfully!

🔐 NGROK AUTHENTICATION SETUP

1. Go to: https://dashboard.ngrok.com/get-started/your-authtoken
2. Copy your authtoken
3. Paste it below

Enter your ngrok authtoken: ··········

🚀 STARTING STREAMLIT APP

✅ Streamlit app is running!
🌐 Public URL: NgrokTunnel: "https://mustachioed-superpiously-joycelyn.ngrok-free.dev" -> "http://localhost:8501"
📱 Share this URL with anyone to access your app!

⚠️ Note: Keep this cell running. Stopping it will close the app.


🛑 Shutting down...
✅ App stopped successfully!
